In [4]:
#Hoeffding's Inequality

import numpy as np
import random
from tqdm import tqdm

In [18]:
def coin_flip_exp():
    experiment = []
    #generate 1000 random coins 
    for i in range(1000):
        coin = []
        #flip each coin ten times
        for k in range(10):
            #define 0 as heads, 1 as tails
            flip = random.randint(0, 1)
            coin.append(flip)
        experiment.append(coin)
    #get first coin    
    c_1 = experiment[0]
    #find nu for first coin
    v_1 = sum(c_1)/10
    #get random coin
    random_index = np.random.choice(1000)
    c_rand = experiment[random_index]
    #find nu for random coin
    v_rand = sum(c_rand)/10
    #get minimum coin
    c_min = min(experiment)
    #find nu for minimum coin
    v_min = sum(c_min)/10
    return v_1, v_rand, v_min

In [19]:
v1_dist = []
vrand_dist = []
vmin_dist = []
#run the coin flip experiment 100,000 times and generate a distribution
for l in tqdm(range(100000)):
    result = coin_flip_exp()
    v1_dist.append(result[0])
    vrand_dist.append(result[1])
    vmin_dist.append(result[2])

100%|██████████| 100000/100000 [36:59<00:00, 45.05it/s]


In [20]:
v_min_avg = np.average(vmin_dist)

In [21]:
print(v_min_avg)

0.04171199999999999


In [161]:
#choose epsilon between 0.1 and 0.5 (reason explained in Problem 2 section)
epsilon_possible = [0.1, 0.2, 0.3, 0.4, 0.5]
dist_list = [v1_dist, vrand_dist, vmin_dist]

for b in range(3):
    print(b) #so that we know which distribution we are checking
    distribution = dist_list[b]
    probability_array = []
    #check the probability that |v-µ| > epsilon is less than or equal to 2e^-2(epsilon^2)*N
    #check for each coin's ten flips
    for epsilon in epsilon_possible:
        counter = 0
        for nu in range(100000):
            if ((np.absolute(distribution[nu] - 0.5)) > epsilon):
                counter += 1
        right_hand_side = 2 * np.exp(-2 * np.square(epsilon) * 10)
        check = counter/100000
        print(epsilon) #so that we know for which epsilons the inequality is satisfied
        if (check <= right_hand_side):
            print(" satisfies Hoeffding")
        else: 
            print(" does not satisfy Hoeffding")

0
0.1
0.34346
1.6374615061559636
 satisfies Hoeffding
0.2
0.11111
0.898657928234443
 satisfies Hoeffding
0.3
0.06684
0.3305977764431731
 satisfies Hoeffding
0.4
0.00208
0.08152440795673238
 satisfies Hoeffding
0.5
0.0
0.013475893998170934
 satisfies Hoeffding
1
0.1
0.34495
1.6374615061559636
 satisfies Hoeffding
0.2
0.10998
0.898657928234443
 satisfies Hoeffding
0.3
0.06564
0.3305977764431731
 satisfies Hoeffding
0.4
0.00185
0.08152440795673238
 satisfies Hoeffding
0.5
0.0
0.013475893998170934
 satisfies Hoeffding
2
0.1
1.0
1.6374615061559636
 satisfies Hoeffding
0.2
0.99925
0.898657928234443
 does not satisfy Hoeffding
0.3
0.9591
0.3305977764431731
 does not satisfy Hoeffding
0.4
0.62453
0.08152440795673238
 does not satisfy Hoeffding
0.5
0.0
0.013475893998170934
 satisfies Hoeffding


In [22]:
#µ = 0.5
#which one satisifes P[v-µ > e]
print(np.average(v1_dist))
print(np.average(vrand_dist))

0.49990299999999993
0.500278


In [23]:
#5

In [162]:
def lin_regression(N):
    N = 100
    #random coordinates chosen to use as our line
    coordinate_one = np.random.uniform(-1, 1, 2)
    coordinate_two = np.random.uniform(-1, 1, 2)

    xN = []
    for i in range(N): 
        #defining the first component as x0 which is always 1
        coordinate = [1]
        #create dataset of N random points
        coordinate[1:2] = np.random.uniform(-1, 1, 2)
        xN.append(coordinate)

    def det_output(points, coordinate_one, coordinate_two):
        #using the determinant to figure out whether the points lie on one side of the line or the other
        #formula: (x-x1)(y2-y1) - (y-y1)(x2-x1) where the point is (x, y)
        #one vector is (x-x1, y-y1), the other is (x2-x1, y2-y1) and we are taking the cross product
        #if the point is on one side of the line, the cross product will be negative, if the point is on the other side of the line, the cross product will be positive
        AD = (points[1] - coordinate_one[0]) * (coordinate_two[1] - coordinate_one[1])
        BC = (points[2] - coordinate_one[1]) * (coordinate_two[0] - coordinate_one[0])
        return np.sign(AD-BC)

    outputs = []
    for x in xN:
        #create a list of outputs using det_output function for each point in dataset
        sign = det_output(x, coordinate_one, coordinate_two)
        outputs.append(int(sign))

    #compute pseudo-inverse
    xTX = np.dot(np.transpose(xN), xN)
    pseudo = np.dot(np.linalg.inv(xTX), np.transpose(xN))
    weights = np.dot(pseudo, outputs)

    incorrect = 0
    correct = 0
    for j in xN:
        #find classification using line
        f_output = det_output(j, coordinate_one, coordinate_two)
        #find classification using weights
        g_output = np.sign(np.dot(weights, j))
        #check accuracy
        if f_output != g_output:
            incorrect +=1
        else:
            correct += 1
    #calculate probability of accuracy
    E_in = incorrect/(incorrect + correct)
    
    #generating 1000 random points for data testing
    points_data = []
    for i in range(1000):
        point = [1]
        point[1:2] = np.random.uniform(-1, 1, 2)
        points_data.append(point)
    incorrect = 0
    correct = 0
    for j in range(1000):
        #find classification using line
        f_output = det_output(points_data[j], coordinate_one, coordinate_two)
        #find classification using weights
        g_output = np.sign(np.dot(weights, points_data[j]))
        #check accuracy
        if f_output != g_output:
            incorrect +=1
        else:
            correct += 1
    #calculate probability of accuracy
    E_out = incorrect/(incorrect + correct)
    
    return E_in, E_out, weights, xN, outputs

In [163]:
all_E_in = []
all_E_out = []
for m in range(1000):
    E_in, E_out, weights, xN, outputs = lin_regression(100)
    all_E_in.append(E_in)
    all_E_out.append(E_out)
print(np.average(E_in))
print(np.average(E_out))

0.01
0.026


In [ ]:
#7

In [70]:
def perceptron(N, weights, xN, outputs):

    #initial weight vector from linear regression
    w = weights

    iteration = 0
    #create a list with indices of all the points that are misclassified based on the given outputs and weights from linear regression
    misclassified = []
    for i in range(N):
        if np.sign(np.dot(w, xN[i])) != outputs[i]:
            misclassified.append(i)
    
    #update weights
    
    while len(misclassified) > 0:
        #pick a random misclassified index
        index = random.choice(misclassified)
        #update weight vector
        w = w + np.dot(outputs[index], xN[index])
        iteration += 1
        #re-update set of misclassified points by checking whether the weight vector results 
        #in the same output
        misclassified = []
        for i in range(N):
            if np.sign(np.dot(w, xN[i])) != outputs[i]:
                misclassified.append(i)
            
    return iteration

In [164]:
#input for number of points to be used
n = 10
iteration_total = []

#run model 1000 times and take the average
for i in range(1000):
    E_in, E_out, weights, xN, outputs = lin_regression(n)
    iter_val = perceptron(n, weights, xN, outputs)
    iteration_total.append(iter_val)

print(np.average(iteration_total))

4.404


In [77]:
#8

In [5]:
#generate training set
points_data = []
for i in range(1000):
    point = [1]
    #generate points
    point[1:2] = np.random.uniform(-1, 1, 2)
    points_data.append(point)
def target_function(point):
    x1 = point[1]
    x2 = point[2]
    return np.sign(np.square(x1) + np.square(x2) - 0.6)

In [6]:
def lin_dataset(data):
    #create array of expected outputs
    output = []
    for item in data:
        output.append(target_function(item))
    #create noise for 10% of points
    for m in range(100):
        rand_point = np.random.choice(1000)
        output[rand_point] = -1 * output[rand_point]
    #compute pseudo-inverse
    xTX = np.dot(np.transpose(data), data)
    pseudo = np.dot(np.linalg.inv(xTX), np.transpose(data))
    weights = np.dot(pseudo, output)

    incorrect = 0
    correct = 0
    for j in data:
        #find classification using line
        f_output = target_function(j)
        #find classification using weights
        g_output = np.sign(np.dot(weights, j))
        #check accuracy
        if f_output != g_output:
            incorrect +=1
        else:
            correct += 1
    #calculate probability of accuracy
    E_in = incorrect/(incorrect + correct)
    
    return E_in, weights

In [99]:
E_in_result = [] 
for a in tqdm(range(1000)):
    E_in_result.append(lin_dataset(points_data)[0])   
print(np.average(E_in_result))

100%|██████████| 1000/1000 [00:29<00:00, 33.38it/s]

0.4821550000000001


In [84]:
#9

In [7]:
transform_data = []
#transform the data
for value in points_data:
    x1 = value[1]
    x2 = value[2]
    transform = [1, x1, x2, x1*x2, np.square(x1), np.square(x2)]
    transform_data.append(transform)
hypothesis = []

for a in tqdm(range(1000)):
    #run the linear regression on the transformed data
    value = lin_dataset(transform_data)
    #get final weights
    hypothesis.append(value[1]) 

100%|██████████| 1000/1000 [00:51<00:00, 19.60it/s]


In [39]:
def check_weights(hypotheses):
    
    weight_a = [-1, 0.05, 0.08, 0.13, 1.5, 1.5]
    weight_b = [-1, 0.05, 0.08, 0.13, 1.5, 15]
    weight_c = [-1, 0.05, 0.08, 0.13, 15, 1.5]
    weight_d = [-1, 1.5, 0.08, 0.13, 0.05, 0.05]
    weight_e = [-1, 0.05, 0.08, 1.5, .15, .15]
    
    given_weights = [weight_a, weight_b, weight_c, weight_d, weight_e]
    
    #create set of random points for testing
    points_random = []
    for i in range(1000):
        point = [1]
        #generate points
        point[1:2] = np.random.uniform(-1, 1, 2)
        point[3:5] = (point[1] * point[2], np.square(point[1]), np.square(point[2]))
        points_random.append(point)
    
    choice_array = []
    for choice in tqdm(given_weights):
        for hypothesis in hypotheses:
            #create array of expected outputs

            incorrect = 0
            correct = 0
            for j in range(1000):
                #find classification using found weights
                f_output = np.sign(np.dot(hypothesis, points_random[j]))
                #find classification using weights
                g_output = np.sign(np.dot(choice, points_random[j]))
                #check accuracy
                if f_output != g_output:
                    incorrect +=1
                else:
                    correct += 1
            #calculate probability of accuracy
            comparison = correct/(incorrect + correct)
            all_comparisons = []
            all_comparisons.append(comparison)
        choice_array.append(np.average(all_comparisons))
    
    return choice_array

In [40]:
weights_comparison = check_weights(hypothesis)


100%|██████████| 5/5 [01:01<00:00, 12.33s/it]


In [41]:
print(weights_comparison)

[0.955, 0.669, 0.654, 0.617, 0.573]


In [45]:
all_E_outs = []
for hypo in hypothesis:    
    #create set of random points for testing
    points_set = []
    for i in range(1000):
        point = [1]
        #generate points
        point[1:2] = np.random.uniform(-1, 1, 2)
        point[3:5] = (point[1] * point[2], np.square(point[1]), np.square(point[2]))
        points_set.append(point)
    f_outputs = []
    #get outputs
    for a in range(1000):
        f_outputs.append(target_function(points_set[a]))
    #add noise
    for m in range(100):
        rand_point = np.random.choice(1000)
        f_outputs[rand_point] = -1 * f_outputs[rand_point]
    incorrect = 0
    correct = 0
    for j in range(1000):
        #find classification using line
        f_output = f_outputs[j]
        #find classification using weights
        g_output = np.sign(np.dot(hypo, points_set[j]))
        #check accuracy
        if f_output != g_output:
            incorrect +=1
        else:
            correct += 1
    #calculate probability of accuracy
    E_out = incorrect/(incorrect + correct)
    all_E_outs.append(E_out)

In [46]:
print(np.average(all_E_outs))

0.115274
